# LCEL Interface

Ngôn ngữ biểu thức LangChain (LangChain Expression Language - LCEL) là một giao diện mạnh mẽ được thiết kế để đơn giản hóa việc tạo và quản lý các chuỗi tùy chỉnh (custom chains) trong LangChain. Nó triển khai giao thức Runnable (Runnable protocol), cung cấp một cách tiêu chuẩn hóa để xây dựng và thực thi các chuỗi mô hình ngôn ngữ (language model chains).


## Environment setup

In [7]:
# Load env
import os
from dotenv import load_dotenv
load_dotenv(override=True, dotenv_path="../.env")

# Check env 
# os.getenv("OPENAI_API_KEY")

True


## LCEL Interface

Để việc tạo các chuỗi tùy chỉnh trở nên dễ dàng nhất có thể, chúng tôi đã triển khai giao thức `Runnable`.

Giao thức `Runnable` được triển khai trong hầu hết các thành phần.

Đây là một giao diện tiêu chuẩn giúp dễ dàng định nghĩa các chuỗi tùy chỉnh và gọi chúng theo cách tiêu chuẩn. Giao diện tiêu chuẩn bao gồm:

* `stream`: Phát luồng một phần phản hồi.
* `invoke`: Gọi một chuỗi trên một đầu vào.
* `batch`: Gọi một chuỗi đối với một danh sách các đầu vào.

Ngoài ra, còn có các phương thức không đồng bộ:

* `astream`: Phát luồng các phần phản hồi một cách không đồng bộ.
* `ainvoke`: Gọi một chuỗi một cách không đồng bộ trên một đầu vào.
* `abatch`: Gọi một chuỗi một cách không đồng bộ đối với một danh sách các đầu vào.
* `astream_log`: Phát luồng phản hồi cuối cùng cũng như các bước trung gian khi chúng xảy ra.

**Giải thích:**

* `Runnable` protocol là một giao diện thống nhất cho phép bạn xây dựng các chuỗi (chains) xử lý dữ liệu một cách linh hoạt.
* Các phương thức `invoke`, `batch`, `stream` và các phiên bản không đồng bộ của chúng cung cấp các cách khác nhau để thực thi các chuỗi này, tùy thuộc vào nhu cầu về đồng bộ, hiệu suất và luồng dữ liệu.
* LCEL(LangChain Expression Language) là một cách để dễ dàng tạo chuỗi bằng cách sử dụng cú pháp đơn giản.

**Log your trace (Ghi lại dấu vết của bạn)**

Chúng tôi cung cấp nhiều cách để ghi lại dấu vết vào LangSmith. Dưới đây, chúng tôi sẽ làm nổi bật cách sử dụng `traceable()`.

Sử dụng mã dưới đây để ghi lại một dấu vết trong LangSmith:

```python
import openai
from langsmith import wrappers, traceable

# Auto-trace LLM calls in-context
client = wrappers.wrap_openai(openai.Client())

@traceable # Auto-trace this function
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("Hello, world!")
# Out:  Hello there! How can I assist you today?
```

**Giải thích:**

* LangSmith là một công cụ để theo dõi và gỡ lỗi các ứng dụng LLM.
* `wrappers.wrap_openai(openai.Client())` bọc client của openai, để có thể theo dõi các lời gọi đến openai.
* `@traceable` là một decorator giúp tự động ghi lại các dấu vết (traces) của hàm `pipeline` vào LangSmith.
* Điều này cho phép bạn kiểm tra các đầu vào, đầu ra và các bước trung gian của chuỗi, giúp bạn hiểu rõ hơn về cách ứng dụng LLM của mình hoạt động.

**Create a chain using LCEL syntax (Tạo một chuỗi sử dụng cú pháp LCEL)**

LCEL syntax là cú pháp để tạo chuỗi trong Langchain. Nó cho phép bạn kết nối các thành phần của LLM một cách dễ dàng.

Ví dụ:

```python
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})
```

Trong ví dụ trên:

* `prompt | model | output_parser` tạo một chuỗi bằng cách kết nối prompt, model và output parser.
* `chain.invoke({"topic": "ice cream"})` gọi chuỗi với đầu vào là `{"topic": "ice cream"}`.

LCEL giúp việc tạo các chuỗi phức tạp trở nên đơn giản và dễ đọc.


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = PromptTemplate.from_template("Describle the {topic} in 3 sentences.")
chain = prompt | model | StrOutputParser()




### stream: real-time output

In [9]:
for token in chain.stream({"topic": "langchain"}):
    print(token, end="", flush=True)


Langchain is a decentralized language learning platform that uses blockchain technology to connect language learners with native speakers for personalized language practice. Users can earn tokens by helping others learn their native language, which can be used to access premium features on the platform. Langchain aims to create a global community of language learners and speakers, breaking down barriers to language learning and promoting cultural exchange.

### invoke

In [10]:
chain.invoke({"topic":"ChatGPT"})

'ChatGPT is an AI-powered chatbot that uses natural language processing to engage in conversations with users. It can provide information, answer questions, and even hold discussions on a wide range of topics. With its advanced language capabilities, ChatGPT aims to provide a more human-like and personalized chat experience.'

### batch

In [11]:
chain.batch([{"topic":"Chat GPT"}, {"topic":"Langchain"}])

['Chat GPT is a state-of-the-art language model developed by OpenAI that can generate human-like responses in conversation. It uses deep learning techniques to understand and generate text based on the input it receives. Chat GPT has been trained on a vast amount of text data to improve its ability to engage in natural and coherent conversations with users.',
 'Langchain is a decentralized platform that allows users to create and manage their own blockchain networks. It provides tools for developers to easily build and deploy custom blockchain applications. Langchain also offers a secure and scalable infrastructure for businesses looking to implement blockchain technology.']

Bạn có thể sử dụng tham số `max_concurrency` để đặt số lượng yêu cầu đồng thời | Từ điển `config` sử dụng khóa `max_concurrency` để đặt số lượng thao tác tối đa có thể được xử lý đồng thời. Ở đây, nó được thiết lập để xử lý tối đa ba công việc đồng thời.


In [12]:
topics =  [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "multimodal"},
        {"topic": "programming"},
        {"topic": "machineLearning"},
    ]

chain.batch(topics, config={"max_concurrency":3})

['ChatGPT is an AI-powered chatbot that uses natural language processing to engage in conversations with users. It can provide information, answer questions, and even hold casual conversations on a wide range of topics. With its advanced language capabilities, ChatGPT aims to provide a more human-like and personalized chat experience for users.',
 'Instagram is a popular social media platform where users can share photos and videos with their followers. It allows users to edit and enhance their photos with filters and other editing tools. Users can also engage with others by liking, commenting, and sharing posts.',
 'Multimodal refers to the use of multiple modes of communication, such as verbal, visual, and gestural, to convey information. It allows for a more comprehensive and engaging way of presenting information, catering to different learning styles and preferences. By incorporating various modes of communication, multimodal communication can enhance understanding and retention o

### async stream

"Hàm `chain.astream` tạo ra một luồng bất đồng bộ (asynchronous stream) và xử lý các tin nhắn cho một chủ đề (topic) cụ thể một cách bất đồng bộ.

Nó sử dụng vòng lặp `async for` để tuần tự nhận các tin nhắn từ luồng, và hàm `print` để in ngay lập tức nội dung của các tin nhắn (`s.content`). `end=""` vô hiệu hóa việc xuống dòng sau khi in, và `flush=True` buộc bộ đệm đầu ra (output buffer) được làm rỗng để đảm bảo in ngay lập tức."


In [14]:
async for token in chain.astream(topics[0]):
    print(token, end="", flush=True)

ChatGPT is an AI-powered chatbot that uses natural language processing to engage in conversations with users. It can provide information, answer questions, and even hold discussions on a wide range of topics. With its advanced language capabilities, ChatGPT aims to simulate human-like interactions and provide personalized responses to users.

### async invoke

Phương thức `ainvoke` của một đối tượng `chain` thực hiện một thao tác bất đồng bộ (asynchronously) với các đối số (arguments) được cung cấp. Phương thức này có thể được sử dụng để yêu cầu xử lý bất đồng bộ cho một chủ đề (topic) cụ thể.


In [19]:
my_process = chain.ainvoke(topics[0])

In [20]:
await my_process

'ChatGPT is an AI-powered chatbot that uses natural language processing to engage in conversations with users. It can provide information, answer questions, and even hold discussions on a wide range of topics. With its advanced language capabilities, ChatGPT aims to create a more interactive and personalized experience for users seeking assistance or entertainment.'

### async batch

The function abatch batches a series of actions asynchronously.

In [21]:
my_batch_process = chain.abatch(topics)

In [22]:
await my_batch_process

['ChatGPT is an AI-powered chatbot that can engage in conversations with users on a wide range of topics. It uses natural language processing to understand and respond to user input in a human-like manner. ChatGPT can provide information, answer questions, and even engage in casual conversations with users.',
 'Instagram is a popular social media platform where users can share photos and videos with their followers. It allows users to edit and enhance their photos with filters and other editing tools. Users can also engage with others by liking, commenting, and sharing posts.',
 'Multimodal refers to the use of multiple modes of communication, such as visual, auditory, and tactile, to convey information. It allows for a more engaging and interactive experience for the audience, as they can access information through different sensory channels. By incorporating various modes of communication, multimodal communication can cater to different learning styles and preferences.',
 'Programmin

### Parallel

Hãy cùng xem cách Ngôn ngữ Biểu thức LangChain (LangChain Expression Language) hỗ trợ các yêu cầu song song. Cụ thể, khi bạn sử dụng `RunnableParallel` (thường được viết dưới dạng từ điển), bạn thực thi từng phần tử song song.

Dưới đây là một ví dụ về việc chạy hai tác vụ song song bằng lớp `RunnableParallel` trong module `langchain_core.runnables`.

1.  **Tạo hai chuỗi (chain1, chain2):**
    * Sử dụng phương thức `ChatPromptTemplate.from_template` để lấy thủ đô và diện tích cho một `country` (quốc gia) nhất định.
    * `ChatPromptTemplate.from_template` là một cách để tạo ra các prompt mẫu cho các mô hình chat.
2.  **Kết nối các chuỗi:**
    * Các chuỗi này được kết nối thông qua `model` (mô hình) và toán tử pipe (`|`) tương ứng.
    * `model` ở đây thường là một instance của LLM.
    * Toán tử pipe (`|`) là một cách viết ngắn gọn để kết nối các `runnable` với nhau, truyền output của `runnable` phía trước vào input của `runnable` phía sau.
3.  **Sử dụng `RunnableParallel`:**
    * Cuối cùng, chúng ta sử dụng lớp `RunnableParallel` để kết hợp hai chuỗi này với các khóa `capital` (thủ đô) và `area` (diện tích) để tạo ra một đối tượng `combined` có thể được chạy song song.
    * `RunnableParallel` sẽ chạy `chain1` và `chain2` đồng thời, và kết quả sẽ được trả về dưới dạng một dictionary với key là `capital` và `area`.


In [23]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("Thành phố của {country} là gì?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country} nổi tiếng với điều gì?")
    | model 
    | StrOutputParser()
)

combined = RunnableParallel(capital=chain1, favorite=chain2)

In [24]:
combined.invoke({"country": "Vietnam"})

{'capital': 'Thủ đô của Việt Nam là Hà Nội.',
 'favorite': 'Vietnam nổi tiếng với nhiều điều, bao gồm:\n\n1. Văn hóa và lịch sử phong phú: Vietnam có một lịch sử lâu đời và văn hóa đa dạng, được thể hiện qua các di sản văn hóa như các di tích lịch sử, ngôn ngữ, trang phục truyền thống, ẩm thực và nghệ thuật dân gian.\n\n2. Phong cảnh đẹp: Vietnam có những cảnh đẹp tự nhiên tuyệt vời như vịnh Hạ Long, sông Mekong, núi rừng Tây Bắc, bãi biển Nha Trang và Đà Nẵng.\n\n3. Ẩm thực đặc sắc: Vietnam nổi tiếng với ẩm thực phong phú, ngon và đa dạng như phở, bánh mì, bún chả, bánh xèo, nem nướng, cà phê và nước mía.\n\n4. Du lịch: Vietnam là điểm đến phổ biến cho du khách quốc tế với các điểm du lịch nổi tiếng như Hà Nội, Sài Gòn, Hội An, Đà Lạt, Huế và Phú Quốc.\n\n5. Người dân thân thiện: Người Việt Nam được biết đến là những người thân thiện, hòa nhã và nhiệt tình đối với du khách.'}

### Parallelism in batches

In [25]:
# Processes the given data in batches.
combined.batch([{"country": "Canada"}, {"country": "USA"}])

[{'capital': 'Thủ đô của Canada là Ottawa.',
  'favorite': 'Canada nổi tiếng với nhiều điều, bao gồm cả cảnh đẹp tự nhiên như hồ Ontario, thác Niagara và Rocky Mountains, cũng như với văn hóa đa dạng và hệ thống giáo dục và y tế hàng đầu thế giới. Ngoài ra, Canada cũng nổi tiếng với việc bảo vệ môi trường và quyền con người, cũng như với các môn thể thao như hockey và curling.'},
 {'capital': 'Có nhiều thành phố ở Hoa Kỳ, một số thành phố lớn và nổi tiếng bao gồm New York City, Los Angeles, Chicago, Houston, Philadelphia, Phoenix, San Antonio, San Diego, Dallas và San Jose.',
  'favorite': 'Mỹ nổi tiếng với nhiều điều, bao gồm:\n\n1. Văn hóa pop: Mỹ là quốc gia sản xuất nhiều phim, âm nhạc, truyền hình và các sản phẩm văn hóa pop nổi tiếng trên toàn thế giới.\n\n2. Công nghệ: Mỹ là một trong những quốc gia hàng đầu về công nghệ và đổi mới, với nhiều công ty công nghệ lớn như Apple, Google, Facebook, Amazon.\n\n3. Thể thao: Mỹ có nhiều môn thể thao phổ biến và nổi tiếng như bóng đá Mỹ, 